In [ ]:
import os
import json
from src.data import CustomDataset, OriginalDataset
from transformers import AutoTokenizer

"""
id | 원본 데이터 | 전처리 데이터 | label | 요약문

"""
# dev data result 넣기
JSON_PATH = "results/dev_result_preprocess.json"
DEV_DATA_PATH = "resource/data/일상대화요약_dev.json"
MODEL_ID = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

with open(JSON_PATH, "r") as f:
    result_data = json.load(f)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
dataset = OriginalDataset(DEV_DATA_PATH, tokenizer)
dataset_preprocessed = CustomDataset(DEV_DATA_PATH, tokenizer)

In [ ]:
dataset # 전처리 데이터 추출

In [ ]:
import pandas as pd
import tqdm

result_df = pd.DataFrame(columns=['id', 'original', 'preprocessed', 'inference', 'label'])

for idx in tqdm.tqdm(range(len(result_data[:5]))):
    result_df.loc[len(result_df), 'id'] = result_data[idx]['id']
    # 전처리 이전
    result_df.loc[len(result_df), 'original'] = dataset[idx]
    # 전처리 이후
    result_df.loc[len(result_df), 'preprocessed'] = dataset_preprocessed[idx]
    result_df.loc[len(result_df), 'inference'] = result_data[idx]['inference']
    result_df.loc[len(result_df), 'label'] = result_data[idx]['output']